# CESM2 Large Ensemble Data Retrieval
In this Jupyter Notebook, we retrieve and coarsen CESM2 Large Ensemble data stored in NCAR's glade. We will take this data and store it somewhere our team can access without having access to NCAR's computing resources.

In [1]:
##### LOADING IN PACKAGES #--------------------------------------------------------------
import s3fs; import xarray as xr; import numpy as np
import pandas as pd; 
import dask.array as da
import ocetrac

import matplotlib.pyplot as plt; import cartopy.crs as ccrs

import warnings; import expectexception
warnings.filterwarnings('ignore')

import netCDF4 as nc; import datetime as dt
import scipy

import intake; import pprint
# Allow multiple lines per cell to be displayed without print (default is just last line)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Enable more explicit control of DataFrame display (e.g., to omit annoying line numbers)
from IPython.display import HTML

In [2]:
# Open original collection description file #----------------------------------------------
cat_url_orig = '/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cesm2-le.json'
coll_orig = intake.open_esm_datastore(cat_url_orig)

In [3]:
coll_orig

,unique
component,6
stream,26
case,200
member_id,100
variable,1906
start_time,157
end_time,180
time_range,163
long_name,1800
units,184


In [4]:
# Show the first few lines of the catalog
coll_orig.df.head(10)

,component,stream,case,member_id,variable,start_time,end_time,time_range,long_name,units,vertical_levels,frequency,path,experiment,forcing_variant,cesm_member_id,control_branch_year,cmip_experiment_id
0,atm,cam.h1,b.e21.BHISTcmip6.f09_g17.LE2-1001.001,r1i1001p1f1,ACTNI,1850-01-01,1859-12-31,18500101-18591231,Average Cloud Top ice number,m-3,1.0,day_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/a...,historical,cmip6,1001.001,1001,CESM2_historical_r1i1001p1f1
1,atm,cam.h1,b.e21.BHISTcmip6.f09_g17.LE2-1001.001,r1i1001p1f1,ACTNI,1860-01-01,1869-12-31,18600101-18691231,Average Cloud Top ice number,m-3,1.0,day_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/a...,historical,cmip6,1001.001,1001,CESM2_historical_r1i1001p1f1
2,atm,cam.h1,b.e21.BHISTcmip6.f09_g17.LE2-1001.001,r1i1001p1f1,ACTNI,1870-01-01,1879-12-31,18700101-18791231,Average Cloud Top ice number,m-3,1.0,day_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/a...,historical,cmip6,1001.001,1001,CESM2_historical_r1i1001p1f1
3,atm,cam.h1,b.e21.BHISTcmip6.f09_g17.LE2-1001.001,r1i1001p1f1,ACTNI,1880-01-01,1889-12-31,18800101-18891231,Average Cloud Top ice number,m-3,1.0,day_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/a...,historical,cmip6,1001.001,1001,CESM2_historical_r1i1001p1f1
4,atm,cam.h1,b.e21.BHISTcmip6.f09_g17.LE2-1001.001,r1i1001p1f1,ACTNI,1890-01-01,1899-12-31,18900101-18991231,Average Cloud Top ice number,m-3,1.0,day_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/a...,historical,cmip6,1001.001,1001,CESM2_historical_r1i1001p1f1
5,atm,cam.h1,b.e21.BHISTcmip6.f09_g17.LE2-1001.001,r1i1001p1f1,ACTNI,1900-01-01,1909-12-31,19000101-19091231,Average Cloud Top ice number,m-3,1.0,day_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/a...,historical,cmip6,1001.001,1001,CESM2_historical_r1i1001p1f1
6,atm,cam.h1,b.e21.BHISTcmip6.f09_g17.LE2-1001.001,r1i1001p1f1,ACTNI,1910-01-01,1919-12-31,19100101-19191231,Average Cloud Top ice number,m-3,1.0,day_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/a...,historical,cmip6,1001.001,1001,CESM2_historical_r1i1001p1f1
7,atm,cam.h1,b.e21.BHISTcmip6.f09_g17.LE2-1001.001,r1i1001p1f1,ACTNI,1920-01-01,1929-12-31,19200101-19291231,Average Cloud Top ice number,m-3,1.0,day_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/a...,historical,cmip6,1001.001,1001,CESM2_historical_r1i1001p1f1
8,atm,cam.h1,b.e21.BHISTcmip6.f09_g17.LE2-1001.001,r1i1001p1f1,ACTNI,1930-01-01,1939-12-31,19300101-19391231,Average Cloud Top ice number,m-3,1.0,day_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/a...,historical,cmip6,1001.001,1001,CESM2_historical_r1i1001p1f1
9,atm,cam.h1,b.e21.BHISTcmip6.f09_g17.LE2-1001.001,r1i1001p1f1,ACTNI,1940-01-01,1949-12-31,19400101-19491231,Average Cloud Top ice number,m-3,1.0,day_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/a...,historical,cmip6,1001.001,1001,CESM2_historical_r1i1001p1f1


In [30]:
# Show expanded version of collection structure with details
import pprint
uniques = coll_orig.unique(columns=["component", "frequency", "experiment", "variable"])
pprint.pprint(uniques, compact=True, indent=4)

{   'component': {   'count': 6,
                     'values': ['lnd', 'rof', 'ocn', 'ice', 'atm', 'glc']},
    'experiment': {'count': 2, 'values': ['historical', 'ssp370']},
    'frequency': {   'count': 7,
                     'values': [   'day_1', 'hour_1', 'hour_6', 'day_365',
                                   'hour_3', 'year_1', 'month_1']},
    'variable': {   'count': 1906,
                    'values': [   'CS_MIXPOSS', 'Jint_100m_NH4', 'PDELDRY',
                                  'O_SCALAR', 'QCHARGE', 'SOIL3C', 'NO3', 'UPV',
                                  'sidmasslat', 'PO4_diat_uptake', 'TSL',
                                  'pom_a4SFWET', 'dvidtd', 'DIA_IMPVF_PO4',
                                  'H2OSFC', 'WT_Fe', 'WT_DIC_ALT_CO2', 'ROFF_F',
                                  'ncl_a3SF', 'ncl_c3', 'tend_zint_100m_DOCr',
                                  'Z200', 'graze_diaz_zint_100m', 'U050',
                                  'CLD_CAL_LIQ', 'SOIL1C_vr', 'ncl_c3SF

In [28]:
# ----------------
# Use https://www.cesm.ucar.edu/projects/community-projects/LENS2/variable-list/ to see 
# where our variables are located
# ----------------

### ----- SST (complete)
# subset_SST = coll_orig.search(component='atm',variable='SST',frequency='month_1',experiment='historical')

## Sea Level Pressure (complete)
# subset_PSL = coll_orig.search(component='atm',variable='PSL',frequency='month_1',experiment='historical')

### ----- SSH (complete on GoogleDrive) # Not sure if this is correct because this is on a POP grid. I did not coarsen the SSH grid.
subset_SSH = coll_orig.search(component='ocn',variable='SSH',frequency='month_1',experiment='historical')
member_id_list = subset_SSH.df.member_id.unique()
print(member_id_list)

### ----- Precipitation (complete)
# subset_precip = coll_orig.search(component='atm',variable='PRECT',frequency='month_1',experiment='historical')

## 10m Wind U (complete)
# subset_10U = coll_orig.search(component='atm',variable='U10',frequency='month_1',experiment='historical')

## 10m Wind V (ran into some problems here!)
subset_10V = coll_orig.search(component='atm',variable='V10',frequency='month_1',experiment='historical')

['r1i1001p1f1' 'r2i1021p1f1' 'r3i1041p1f1' 'r4i1061p1f1' 'r5i1081p1f1'
 'r6i1101p1f1' 'r7i1121p1f1' 'r8i1141p1f1' 'r9i1161p1f1' 'r10i1181p1f1'
 'r1i1231p1f1' 'r2i1231p1f1' 'r3i1231p1f1' 'r4i1231p1f1' 'r5i1231p1f1'
 'r6i1231p1f1' 'r7i1231p1f1' 'r8i1231p1f1' 'r9i1231p1f1' 'r10i1231p1f1'
 'r1i1251p1f1' 'r2i1251p1f1' 'r3i1251p1f1' 'r4i1251p1f1' 'r5i1251p1f1'
 'r6i1251p1f1' 'r7i1251p1f1' 'r8i1251p1f1' 'r9i1251p1f1' 'r10i1251p1f1'
 'r1i1281p1f1' 'r2i1281p1f1' 'r3i1281p1f1' 'r4i1281p1f1' 'r5i1281p1f1'
 'r6i1281p1f1' 'r7i1281p1f1' 'r8i1281p1f1' 'r9i1281p1f1' 'r10i1281p1f1'
 'r1i1301p1f1' 'r2i1301p1f1' 'r3i1301p1f1' 'r4i1301p1f1' 'r5i1301p1f1'
 'r6i1301p1f1' 'r7i1301p1f1' 'r8i1301p1f1' 'r9i1301p1f1' 'r10i1301p1f1'
 'r1i1011p1f2' 'r2i1031p1f2' 'r3i1051p1f2' 'r4i1071p1f2' 'r5i1091p1f2'
 'r6i1111p1f2' 'r7i1131p1f2' 'r8i1151p1f2' 'r9i1171p1f2' 'r10i1191p1f2'
 'r11i1231p1f2' 'r12i1231p1f2' 'r13i1231p1f2' 'r14i1231p1f2'
 'r15i1231p1f2' 'r16i1231p1f2' 'r17i1231p1f2' 'r18i1231p1f2'
 'r19i1231p1f2' 'r20

In [18]:
for i in range(0,10):
    subset = coll_orig.search(component='ocn',variable='SSH',frequency='month_1',experiment='historical',member_id= str(member_id_list[i]))
    dsets = subset.to_dataset_dict(zarr_kwargs={"consolidated": True}, storage_options={"anon": True})
    print(dsets)
    ds = dsets['ocn.historical.pop.h.cmip6.SSH']
    SSH = ds.SSH.isel(member_id=0)
    # SSH_coarse=SSH.coarsen(nlon=3, nlat=3, boundary="trim").mean()
    SSH.load()
    SSH.to_netcdf('/glade/work/cassiacai/ENSO_Project_SSH_'+str(member_id_list[i])+'.nc')


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream.forcing_variant.variable'


{'ocn.historical.pop.h.cmip6.SSH': <xarray.Dataset>
Dimensions:                 (moc_comp: 3, transport_comp: 5, transport_reg: 2, z_t: 60, z_t_150m: 15, z_w: 60, z_w_top: 60, z_w_bot: 60, lat_aux_grid: 395, moc_z: 61, nlat: 384, nlon: 320, time: 1980, d2: 2, member_id: 1)
Coordinates: (12/13)
  * z_t                     (z_t) float32 500.0 1.5e+03 ... 5.125e+05 5.375e+05
  * z_t_150m                (z_t_150m) float32 500.0 1.5e+03 ... 1.45e+04
  * z_w                     (z_w) float32 0.0 1e+03 2e+03 ... 5e+05 5.25e+05
  * z_w_top                 (z_w_top) float32 0.0 1e+03 2e+03 ... 5e+05 5.25e+05
  * z_w_bot                 (z_w_bot) float32 1e+03 2e+03 ... 5.25e+05 5.5e+05
  * lat_aux_grid            (lat_aux_grid) float32 -79.49 -78.95 ... 89.47 90.0
    ...                      ...
    ULONG                   (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    ULAT                    (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
   

<xarray.DataArray 'SSH' (time: 1980, nlat: 384, nlon: 320)>
array([[[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [-176.30836, -175.8463 , -176.16678, ...,        nan,
                nan,        nan],
        ...,
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [-176.49557, -176.15474, -176.47441, ...,        nan,
                nan,        nan],
...
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [-178.06822, -177.02254, -178.08104, ...,        nan,
                nan,        nan],
        ...,
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]]], dtype=float32)
Coordinates:
    ULONG      (nlat, nlon) float64 321.1 322.3 323.4 ... 319.2 319.6 320.0
    ULAT       (nlat, nlon) float64 -78.95 -78.95 -78.95 ... 72.42 72.41 72.41
    TLONG      (nlat, nlon) float64 320.6 321.7 322.8 ... 318.9 319.4 319.8
    TLAT       (nlat, nlon) float64 -79.22 -79.22 -79.22 ... 72.2 72.19 72.19
  * time       (time) object 1850-02-01 00:00:00 ... 2015-01-01 00:00:00
    member_id  <U11 'r1i1001p1f1'
Dimensions without coordinates: nlat, nlon
Attributes:
    long_name:     Sea Surface Height
    units:         centimeter
    grid_loc:      2110
    cell_methods:  time: mean


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream.forcing_variant.variable'


{'ocn.historical.pop.h.cmip6.SSH': <xarray.Dataset>
Dimensions:                 (moc_comp: 3, transport_comp: 5, transport_reg: 2, z_t: 60, z_t_150m: 15, z_w: 60, z_w_top: 60, z_w_bot: 60, lat_aux_grid: 395, moc_z: 61, nlat: 384, nlon: 320, time: 1980, d2: 2, member_id: 1)
Coordinates: (12/13)
  * z_t                     (z_t) float32 500.0 1.5e+03 ... 5.125e+05 5.375e+05
  * z_t_150m                (z_t_150m) float32 500.0 1.5e+03 ... 1.45e+04
  * z_w                     (z_w) float32 0.0 1e+03 2e+03 ... 5e+05 5.25e+05
  * z_w_top                 (z_w_top) float32 0.0 1e+03 2e+03 ... 5e+05 5.25e+05
  * z_w_bot                 (z_w_bot) float32 1e+03 2e+03 ... 5.25e+05 5.5e+05
  * lat_aux_grid            (lat_aux_grid) float32 -79.49 -78.95 ... 89.47 90.0
    ...                      ...
    ULONG                   (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    ULAT                    (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
   

...

{'ocn.historical.pop.h.cmip6.SSH': <xarray.Dataset>
Dimensions:                 (moc_comp: 3, transport_comp: 5, transport_reg: 2, z_t: 60, z_t_150m: 15, z_w: 60, z_w_top: 60, z_w_bot: 60, lat_aux_grid: 395, moc_z: 61, nlat: 384, nlon: 320, time: 1980, d2: 2, member_id: 1)
Coordinates: (12/13)
  * z_t                     (z_t) float32 500.0 1.5e+03 ... 5.125e+05 5.375e+05
  * z_t_150m                (z_t_150m) float32 500.0 1.5e+03 ... 1.45e+04
  * z_w                     (z_w) float32 0.0 1e+03 2e+03 ... 5e+05 5.25e+05
  * z_w_top                 (z_w_top) float32 0.0 1e+03 2e+03 ... 5e+05 5.25e+05
  * z_w_bot                 (z_w_bot) float32 1e+03 2e+03 ... 5.25e+05 5.5e+05
  * lat_aux_grid            (lat_aux_grid) float32 -79.49 -78.95 ... 89.47 90.0
    ...                      ...
    ULONG                   (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    ULAT                    (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
   

<xarray.DataArray 'SSH' (time: 1980, nlat: 384, nlon: 320)>
array([[[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [-174.5281 , -174.3522 , -174.4432 , ...,        nan,
                nan,        nan],
        ...,
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [-174.72598, -173.97714, -174.52818, ...,        nan,
                nan,        nan],
...
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [-176.74213, -176.17902, -176.74323, ...,        nan,
                nan,        nan],
        ...,
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]]], dtype=float32)
Coordinates:
    ULONG      (nlat, nlon) float64 321.1 322.3 323.4 ... 319.2 319.6 320.0
    ULAT       (nlat, nlon) float64 -78.95 -78.95 -78.95 ... 72.42 72.41 72.41
    TLONG      (nlat, nlon) float64 320.6 321.7 322.8 ... 318.9 319.4 319.8
    TLAT       (nlat, nlon) float64 -79.22 -79.22 -79.22 ... 72.2 72.19 72.19
  * time       (time) object 1850-02-01 00:00:00 ... 2015-01-01 00:00:00
    member_id  <U11 'r9i1161p1f1'
Dimensions without coordinates: nlat, nlon
Attributes:
    long_name:     Sea Surface Height
    units:         centimeter
    grid_loc:      2110
    cell_methods:  time: mean


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream.forcing_variant.variable'


{'ocn.historical.pop.h.cmip6.SSH': <xarray.Dataset>
Dimensions:                 (moc_comp: 3, transport_comp: 5, transport_reg: 2, z_t: 60, z_t_150m: 15, z_w: 60, z_w_top: 60, z_w_bot: 60, lat_aux_grid: 395, moc_z: 61, nlat: 384, nlon: 320, time: 1980, d2: 2, member_id: 1)
Coordinates: (12/13)
  * z_t                     (z_t) float32 500.0 1.5e+03 ... 5.125e+05 5.375e+05
  * z_t_150m                (z_t_150m) float32 500.0 1.5e+03 ... 1.45e+04
  * z_w                     (z_w) float32 0.0 1e+03 2e+03 ... 5e+05 5.25e+05
  * z_w_top                 (z_w_top) float32 0.0 1e+03 2e+03 ... 5e+05 5.25e+05
  * z_w_bot                 (z_w_bot) float32 1e+03 2e+03 ... 5.25e+05 5.5e+05
  * lat_aux_grid            (lat_aux_grid) float32 -79.49 -78.95 ... 89.47 90.0
    ...                      ...
    ULONG                   (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    ULAT                    (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
   

<xarray.DataArray 'SSH' (time: 1980, nlat: 384, nlon: 320)>
array([[[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [-175.55447, -174.75598, -175.52606, ...,        nan,
                nan,        nan],
        ...,
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [-174.4318 , -174.02519, -174.34256, ...,        nan,
                nan,        nan],
...
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [-181.47354, -180.33519, -181.48592, ...,        nan,
                nan,        nan],
        ...,
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]]], dtype=float32)
Coordinates:
    ULONG      (nlat, nlon) float64 321.1 322.3 323.4 ... 319.2 319.6 320.0
    ULAT       (nlat, nlon) float64 -78.95 -78.95 -78.95 ... 72.42 72.41 72.41
    TLONG      (nlat, nlon) float64 320.6 321.7 322.8 ... 318.9 319.4 319.8
    TLAT       (nlat, nlon) float64 -79.22 -79.22 -79.22 ... 72.2 72.19 72.19
  * time       (time) object 1850-02-01 00:00:00 ... 2015-01-01 00:00:00
    member_id  <U12 'r10i1181p1f1'
Dimensions without coordinates: nlat, nlon
Attributes:
    long_name:     Sea Surface Height
    units:         centimeter
    grid_loc:      2110
    cell_methods:  time: mean

In [21]:
for i in range(0,10):
    print(i)
    subset = coll_orig.search(component='atm',variable='PRECT',frequency='month_1',experiment='historical',member_id= str(member_id_list[i]))
    dsets = subset.to_dataset_dict(zarr_kwargs={"consolidated": True}, storage_options={"anon": True})
    ds = dsets['atm.historical.cam.h0.cmip6.PRECT']
    PRECT = ds.PRECT.isel(member_id=0)
    PRECT_coarse=PRECT.coarsen(lon=3, lat=3, boundary="trim").mean()
    PRECT_coarse.load()
    PRECT_coarse.to_netcdf('/glade/work/cassiacai/ENSO_Project_PRECT_'+str(member_id_list[i])+'.nc')

0

--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream.forcing_variant.variable'


<xarray.DataArray 'PRECT' (time: 1980, lat: 64, lon: 96)>
array([[[3.48715345e-09, 3.77534537e-09, 4.03973210e-09, ...,
         3.47869089e-09, 3.50309337e-09, 3.50341534e-09],
        [2.72280642e-09, 2.66936118e-09, 2.87544966e-09, ...,
         2.59223221e-09, 2.65122013e-09, 2.85544943e-09],
        [2.63866196e-09, 2.28535657e-09, 1.90262828e-09, ...,
         2.66542544e-09, 2.97676062e-09, 2.94903812e-09],
        ...,
        [6.09709394e-09, 5.59050939e-09, 5.03952036e-09, ...,
         6.60631816e-09, 6.51700560e-09, 6.47268150e-09],
        [3.51417651e-09, 3.32560801e-09, 3.37665496e-09, ...,
         4.43254367e-09, 4.17558876e-09, 3.85297438e-09],
        [2.17601981e-09, 2.20173346e-09, 2.22835750e-09, ...,
         2.10674056e-09, 2.12717755e-09, 2.15121521e-09]],

       [[1.82377435e-09, 1.73516268e-09, 1.67118819e-09, ...,
         1.93229344e-09, 1.89954075e-09, 1.86806015e-09],
        [2.65283107e-09, 2.51350496e-09, 2.49326404e-09, ...,
         2.64837796e-09, 2.90910318e-09, 2.85490831e-09],
        [3.91579213e-09, 3.41311890e-09, 2.71574452e-09, ...,
         4.99017450e-09, 4.73722706e-09, 4.44167769e-09],
...
        [6.11154238e-09, 5.93121818e-09, 5.87240079e-09, ...,
         6.53425269e-09, 6.04731776e-09, 5.99632521e-09],
        [6.95693814e-09, 7.18574000e-09, 7.21143634e-09, ...,
         5.40081313e-09, 6.09807405e-09, 6.64802657e-09],
        [4.64228611e-09, 4.66143746e-09, 4.67239092e-09, ...,
         4.66839367e-09, 4.64056793e-09, 4.63162486e-09]],

       [[2.93139601e-09, 2.90068258e-09, 2.86594215e-09, ...,
         3.02711212e-09, 2.98982727e-09, 2.96087621e-09],
        [3.08060422e-09, 2.88360180e-09, 2.71718248e-09, ...,
         3.67223096e-09, 3.44505779e-09, 3.29812710e-09],
        [3.86018506e-09, 3.53231155e-09, 3.20748317e-09, ...,
         5.30219957e-09, 4.87533036e-09, 4.28177716e-09],
        ...,
        [1.35067530e-08, 1.18246826e-08, 1.14390453e-08, ...,
         2.73137459e-08, 2.17710650e-08, 1.68746457e-08],
        [1.01043591e-08, 9.21344689e-09, 8.37197867e-09, ...,
         1.20328361e-08, 1.14225553e-08, 1.07905702e-08],
        [6.96626135e-09, 6.72029188e-09, 6.47983178e-09, ...,
         7.45775264e-09, 7.33184580e-09, 7.17368698e-09]]], dtype=float32)
Coordinates:
  * lat        (lat) float64 -89.06 -86.23 -83.4 -80.58 ... 83.4 86.23 89.06
  * lon        (lon) float64 1.25 5.0 8.75 12.5 ... 346.2 350.0 353.8 357.5
  * time       (time) object 1850-02-01 00:00:00 ... 2015-01-01 00:00:00
    member_id  <U11 'r1i1001p1f1'
Attributes:
    units:         m/s
    long_name:     Total (convective and large-scale) precipitation rate (liq...
    cell_methods:  time: mean

1

--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream.forcing_variant.variable'


<xarray.DataArray 'PRECT' (time: 1980, lat: 64, lon: 96)>
array([[[1.35249323e-09, 1.33935119e-09, 1.32768663e-09, ...,
         1.37894185e-09, 1.36845069e-09, 1.35969447e-09],
        [6.08713580e-10, 6.20322960e-10, 6.26660557e-10, ...,
         5.37249523e-10, 5.48313006e-10, 5.80238413e-10],
        [2.35843373e-10, 2.70068995e-10, 3.23555793e-10, ...,
         1.43432294e-10, 1.69381259e-10, 2.05542278e-10],
        ...,
        [1.34608547e-08, 1.28269697e-08, 1.21050743e-08, ...,
         9.83127713e-09, 1.18400942e-08, 1.32059634e-08],
        [1.20533930e-08, 1.19069954e-08, 1.13993712e-08, ...,
         1.14287522e-08, 1.19780568e-08, 1.20515375e-08],
        [1.18912373e-08, 1.19469883e-08, 1.19622339e-08, ...,
         1.16485861e-08, 1.17547900e-08, 1.18282673e-08]],

       [[9.02189434e-10, 8.79069262e-10, 8.54125048e-10, ...,
         8.57295290e-10, 8.74529504e-10, 8.99037400e-10],
        [1.11240850e-09, 1.08072351e-09, 1.05990816e-09, ...,
         1.30255551e-09, 1.20914900e-09, 1.14444099e-09],
        [2.60255262e-09, 2.40393838e-09, 2.36924058e-09, ...,
         2.69213718e-09, 2.58240518e-09, 2.63440381e-09],
...
        [1.20855379e-08, 1.31141240e-08, 1.34300553e-08, ...,
         8.27225666e-09, 9.48990930e-09, 1.07516795e-08],
        [8.50092796e-09, 8.85859208e-09, 9.02887365e-09, ...,
         7.17811188e-09, 7.56737251e-09, 8.06464406e-09],
        [4.21299040e-09, 4.23108748e-09, 4.22209911e-09, ...,
         4.15234602e-09, 4.17351798e-09, 4.19221990e-09]],

       [[7.10903281e-10, 7.11653181e-10, 7.19312054e-10, ...,
         7.45038142e-10, 7.33841765e-10, 7.18830606e-10],
        [1.32461042e-09, 1.22832877e-09, 1.25105393e-09, ...,
         1.38666756e-09, 1.38224210e-09, 1.32574218e-09],
        [1.61438030e-09, 1.68289638e-09, 2.34199593e-09, ...,
         2.04687511e-09, 1.95323269e-09, 1.83455973e-09],
        ...,
        [8.75220341e-09, 9.26411658e-09, 1.01183080e-08, ...,
         9.98497018e-09, 9.05211017e-09, 8.65599770e-09],
        [7.26025862e-09, 7.59110819e-09, 7.82198928e-09, ...,
         6.18335516e-09, 6.59050015e-09, 6.91207935e-09],
        [4.04096179e-09, 4.05020506e-09, 4.03929290e-09, ...,
         3.88686017e-09, 3.95083166e-09, 4.00340205e-09]]], dtype=float32)
Coordinates:
  * lat        (lat) float64 -89.06 -86.23 -83.4 -80.58 ... 83.4 86.23 89.06
  * lon        (lon) float64 1.25 5.0 8.75 12.5 ... 346.2 350.0 353.8 357.5
  * time       (time) object 1850-02-01 00:00:00 ... 2015-01-01 00:00:00
    member_id  <U11 'r2i1021p1f1'
Attributes:
    units:         m/s
    long_name:     Total (convective and large-scale) precipitation rate (liq...
    cell_methods:  time: mean

2

--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream.forcing_variant.variable'


...

<xarray.DataArray 'PRECT' (time: 1980, lat: 64, lon: 96)>
array([[[4.67555272e-10, 4.76415574e-10, 4.64420696e-10, ...,
         5.03447894e-10, 4.79299489e-10, 4.66618022e-10],
        [1.47811222e-10, 1.43633674e-10, 1.82078005e-10, ...,
         2.12064880e-10, 1.85433460e-10, 1.71395440e-10],
        [5.55195279e-10, 6.47433163e-10, 4.21671920e-10, ...,
         3.57804175e-10, 3.87436805e-10, 4.13562129e-10],
        ...,
        [6.20991347e-09, 6.35579678e-09, 6.04649664e-09, ...,
         5.46026024e-09, 5.58880409e-09, 5.87894666e-09],
        [5.08598319e-09, 4.83593565e-09, 4.51896431e-09, ...,
         5.08278886e-09, 5.16815524e-09, 5.17176435e-09],
        [3.72244902e-09, 3.68543196e-09, 3.64052855e-09, ...,
         3.79626330e-09, 3.77474807e-09, 3.75099063e-09]],

       [[2.04238160e-09, 2.10314099e-09, 2.15633356e-09, ...,
         1.97625827e-09, 1.99228167e-09, 2.02154626e-09],
        [1.13422916e-09, 1.42705925e-09, 1.58375135e-09, ...,
         1.15956245e-09, 1.11807752e-09, 1.12079412e-09],
        [7.43352657e-10, 9.06429098e-10, 1.11182752e-09, ...,
         6.46190546e-10, 6.66583844e-10, 6.78439749e-10],
...
        [1.62558447e-08, 1.56080553e-08, 1.50410813e-08, ...,
         1.28293642e-08, 1.50643427e-08, 1.63555480e-08],
        [1.60391753e-08, 1.72423960e-08, 1.77769408e-08, ...,
         1.32586582e-08, 1.40728265e-08, 1.49457087e-08],
        [1.52594613e-08, 1.53356456e-08, 1.53060142e-08, ...,
         1.45713459e-08, 1.48161394e-08, 1.50558499e-08]],

       [[1.52138879e-09, 1.48141133e-09, 1.47397905e-09, ...,
         1.75390624e-09, 1.66301817e-09, 1.57933977e-09],
        [5.88978200e-10, 5.16699128e-10, 4.49312143e-10, ...,
         1.47292489e-09, 1.08985942e-09, 7.60203067e-10],
        [2.90941021e-10, 1.94790517e-10, 1.42435827e-10, ...,
         9.66516422e-10, 6.82088053e-10, 4.31412739e-10],
        ...,
        [1.14134515e-08, 1.06110214e-08, 9.92020421e-09, ...,
         1.39102445e-08, 1.29269191e-08, 1.22634720e-08],
        [3.66187791e-09, 3.27900751e-09, 2.98982195e-09, ...,
         4.47603021e-09, 4.29076330e-09, 4.00925693e-09],
        [2.60873834e-09, 2.60785882e-09, 2.61342237e-09, ...,
         2.65917910e-09, 2.64023758e-09, 2.61830646e-09]]], dtype=float32)
Coordinates:
  * lat        (lat) float64 -89.06 -86.23 -83.4 -80.58 ... 83.4 86.23 89.06
  * lon        (lon) float64 1.25 5.0 8.75 12.5 ... 346.2 350.0 353.8 357.5
  * time       (time) object 1850-02-01 00:00:00 ... 2015-01-01 00:00:00
    member_id  <U11 'r8i1141p1f1'
Attributes:
    units:         m/s
    long_name:     Total (convective and large-scale) precipitation rate (liq...
    cell_methods:  time: mean

8

--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream.forcing_variant.variable'


<xarray.DataArray 'PRECT' (time: 1980, lat: 64, lon: 96)>
array([[[2.60271554e-10, 2.59141458e-10, 2.55033134e-10, ...,
         2.66569600e-10, 2.65615585e-10, 2.63066208e-10],
        [3.65800418e-10, 3.96831401e-10, 4.81767515e-10, ...,
         4.07640949e-10, 3.99354827e-10, 3.47958690e-10],
        [6.24223173e-10, 7.56834373e-10, 8.23394020e-10, ...,
         1.02805564e-09, 7.60542074e-10, 5.35334221e-10],
        ...,
        [4.66105687e-09, 5.70097747e-09, 7.55924656e-09, ...,
         5.94114713e-09, 5.08518383e-09, 4.48928450e-09],
        [2.48002174e-09, 2.49354115e-09, 2.58718469e-09, ...,
         2.79545476e-09, 2.60459521e-09, 2.51100474e-09],
        [2.87677882e-09, 2.87314039e-09, 2.87987723e-09, ...,
         2.92757285e-09, 2.91077518e-09, 2.89294544e-09]],

       [[2.57295807e-09, 2.54215204e-09, 2.50757171e-09, ...,
         2.65472067e-09, 2.62880762e-09, 2.60376476e-09],
        [2.36537145e-09, 2.27471952e-09, 2.16178520e-09, ...,
         2.80845192e-09, 2.60489341e-09, 2.43658893e-09],
        [3.22586757e-09, 2.91793012e-09, 2.58169552e-09, ...,
         3.97421873e-09, 3.64942410e-09, 3.33562555e-09],
...
        [7.11543890e-09, 7.40117878e-09, 7.67769492e-09, ...,
         8.72675177e-09, 7.94091903e-09, 7.19491533e-09],
        [5.41841016e-09, 5.76188963e-09, 5.97320993e-09, ...,
         4.52322180e-09, 4.72056172e-09, 5.04762276e-09],
        [4.89378094e-09, 5.11006615e-09, 5.31745803e-09, ...,
         4.49059456e-09, 4.59600047e-09, 4.72318584e-09]],

       [[3.58572905e-09, 3.48396600e-09, 3.39625972e-09, ...,
         3.89416943e-09, 3.80920717e-09, 3.69847597e-09],
        [1.83442539e-09, 1.68981551e-09, 1.52873070e-09, ...,
         2.98432323e-09, 2.38495690e-09, 2.05452544e-09],
        [1.10694254e-09, 8.82053819e-10, 5.60757885e-10, ...,
         1.89841676e-09, 1.61511160e-09, 1.33485500e-09],
        ...,
        [1.35507872e-08, 1.22336097e-08, 1.09352918e-08, ...,
         1.74809021e-08, 1.67955889e-08, 1.50985446e-08],
        [1.27988722e-08, 1.18985621e-08, 1.10908296e-08, ...,
         1.50087764e-08, 1.45934616e-08, 1.37381200e-08],
        [1.01219779e-08, 1.00367252e-08, 9.93117055e-09, ...,
         1.01443254e-08, 1.01568025e-08, 1.01732143e-08]]], dtype=float32)
Coordinates:
  * lat        (lat) float64 -89.06 -86.23 -83.4 -80.58 ... 83.4 86.23 89.06
  * lon        (lon) float64 1.25 5.0 8.75 12.5 ... 346.2 350.0 353.8 357.5
  * time       (time) object 1850-02-01 00:00:00 ... 2015-01-01 00:00:00
    member_id  <U11 'r9i1161p1f1'
Attributes:
    units:         m/s
    long_name:     Total (convective and large-scale) precipitation rate (liq...
    cell_methods:  time: mean

9

--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream.forcing_variant.variable'


<xarray.DataArray 'PRECT' (time: 1980, lat: 64, lon: 96)>
array([[[1.2936435e-09, 1.3208721e-09, 1.3873068e-09, ...,
         1.2868325e-09, 1.2862484e-09, 1.2864696e-09],
        [8.5540425e-10, 9.1846952e-10, 1.0154839e-09, ...,
         9.8579322e-10, 9.1353614e-10, 8.7327046e-10],
        [3.4734582e-10, 3.1381392e-10, 3.8001924e-10, ...,
         5.6341398e-10, 4.6540449e-10, 4.2286163e-10],
        ...,
        [8.1240801e-09, 8.3635170e-09, 8.3486649e-09, ...,
         7.6194073e-09, 7.7329023e-09, 7.8191302e-09],
        [6.6741390e-09, 6.6818240e-09, 6.6876029e-09, ...,
         6.5378591e-09, 6.6672197e-09, 6.7185479e-09],
        [5.8590093e-09, 5.8789671e-09, 5.8920233e-09, ...,
         5.7684373e-09, 5.8030025e-09, 5.8329457e-09]],

       [[2.8303815e-09, 2.8495868e-09, 2.8989124e-09, ...,
         2.7072891e-09, 2.7452101e-09, 2.8049312e-09],
        [2.4331166e-09, 2.3414184e-09, 2.3191296e-09, ...,
         2.9876390e-09, 2.7867544e-09, 2.5365035e-09],
        [1.9906043e-09, 1.8424841e-09, 1.7446435e-09, ...,
         2.3942779e-09, 2.2595810e-09, 2.1827087e-09],
...
        [7.8174827e-09, 7.6234850e-09, 7.4776221e-09, ...,
         6.5801427e-09, 7.0457795e-09, 7.6838704e-09],
        [5.5779119e-09, 5.4890923e-09, 5.4373879e-09, ...,
         5.7384701e-09, 5.6486460e-09, 5.6888916e-09],
        [4.6233364e-09, 4.6140354e-09, 4.6175290e-09, ...,
         4.7648325e-09, 4.7287676e-09, 4.6670428e-09]],

       [[4.2079242e-09, 4.1747721e-09, 4.1349799e-09, ...,
         4.4154178e-09, 4.3429451e-09, 4.2754413e-09],
        [4.2170996e-09, 3.9470383e-09, 3.7215147e-09, ...,
         5.2525166e-09, 4.9069722e-09, 4.5513833e-09],
        [3.6758676e-09, 3.2320571e-09, 2.8451008e-09, ...,
         6.2516978e-09, 5.0788511e-09, 4.2203832e-09],
        ...,
        [9.4095327e-09, 1.0546512e-08, 1.1300071e-08, ...,
         8.7287093e-09, 8.7748937e-09, 8.7375911e-09],
        [5.6636464e-09, 5.8228786e-09, 6.0120540e-09, ...,
         5.0545359e-09, 5.1681224e-09, 5.4095111e-09],
        [5.0485820e-09, 5.1491309e-09, 5.2250897e-09, ...,
         4.8544417e-09, 4.8943138e-09, 4.9403206e-09]]], dtype=float32)
Coordinates:
  * lat        (lat) float64 -89.06 -86.23 -83.4 -80.58 ... 83.4 86.23 89.06
  * lon        (lon) float64 1.25 5.0 8.75 12.5 ... 346.2 350.0 353.8 357.5
  * time       (time) object 1850-02-01 00:00:00 ... 2015-01-01 00:00:00
    member_id  <U12 'r10i1181p1f1'
Attributes:
    units:         m/s
    long_name:     Total (convective and large-scale) precipitation rate (liq...
    cell_methods:  time: mean

In [29]:
for i in range(0,10):
    print(i)
    subset = coll_orig.search(component='atm',variable='V10',frequency='month_1',experiment='historical',member_id= str(member_id_list[i]))
    dsets = subset.to_dataset_dict(zarr_kwargs={"consolidated": True}, storage_options={"anon": True})
    print(dsets)
#     ds = dsets['atm.historical.cam.h0.cmip6.V010']
#     V010 = ds.V010.isel(member_id=0)
#     V010_coarse=V010.coarsen(lon=3, lat=3, boundary="trim").mean()
#     V010_coarse.load()
#     V010_coarse.to_netcdf('/glade/work/cassiacai/ENSO_Project_V010_'+str(member_id_list[i])+'.nc')
    

0
{}
1
{}
2
{}
3
{}
4
{}
5
{}
6
{}
7
{}
8
{}
9
{}
